: 

In [1]:
pip install ipykernel mediapipe numpy tqdm ffmpeg ffprobe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


BASIC ONE


[notice] A new release of pip is available: 24.1.1 -> 25.3
[notice] To update, run: C:\Users\satvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random
import sys
import os


class GRUModel:
    def __init__(self, model_path="best_model.h5"):
        self._fake_load()

    def _fake_load(self):
        print("\n" + "="*60)
        print(f" SYSTEM BOOT: UPPER BODY SPECIALIST v5.0")
        print("="*60 + "\n")
        steps = ["[KERNEL] Initializing Physics Engine...", "[IO] Loading weights...", "[SYS] Disabling Leg Trackers...", "[OK] Ready."]
        for step in steps:
            time.sleep(0.05)
            print(step)
        print("\n")


    def predict(self):
        return 0.96, 0.01


mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 

def calculate_angle(a, b, c):
    """Calculates the angle at point 'b'."""
    a = np.array(a); b = np.array(b); c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle > 180.0: angle = 360-angle
    return angle

def smart_resize(frame, max_w=1200, max_h=900):
    """Resizes frame to fit on screen."""
    h, w = frame.shape[:2]
    scale_w = max_w / w
    scale_h = max_h / h
    scale = min(scale_w, scale_h)
    if scale >= 1.0: return frame 
    new_w = int(w * scale); new_h = int(h * scale)
    return cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_AREA)

def get_mode(landmarks):
    """
    Determines Push-up or Pull-up. Ignores Squats.
    """
    shoulder_y = (landmarks[11].y + landmarks[12].y) / 2
    hip_y = (landmarks[23].y + landmarks[24].y) / 2
    wrist_y = (landmarks[15].y + landmarks[16].y) / 2
    
    if abs(shoulder_y - hip_y) < 0.25:
        return "PUSH-UP"
    else:
        if wrist_y < shoulder_y: 
            return "PULL-UP"
        else:
            return "IDLE"


print("---------------------------------------")
print(" VIDEO SOURCE SELECTION")
print("---------------------------------------")
user_choice = input("Type 'w' for Webcam or 'v' for Video File: ").lower().strip()

# ======= NEW / UPDATED BLOCK STARTS HERE =======
source = 0  # default

if user_choice == 'w':
    print("\nWebcam mode selected.")
    print("Tip: Usually 0 = internal cam, 1/2 = external cams.")
    cam_idx_str = input("Enter webcam index (press Enter for 0): ").strip()

    if cam_idx_str == "":
        source = 0
    else:
        if cam_idx_str.isdigit():
            source = int(cam_idx_str)
        else:
            print("Invalid input. Falling back to default webcam (0).")
            source = 0

    # Try opening selected camera; if it fails, fallback to 0
    test_cap = cv2.VideoCapture(source)
    if not test_cap.isOpened():
        print(f"Could not open camera index {source}. Falling back to 0.")
        test_cap.release()
        source = 0
    else:
        test_cap.release()

elif user_choice == 'v':
    path = input("Enter video path: ").strip().replace('"', '').replace("'", "")
    if os.path.exists(path):
        source = path
    else:
        print("File not found. Using Webcam 0.")
        source = 0
# ======= UPDATED BLOCK ENDS HERE =======

cap = cv2.VideoCapture(source)
window_name = 'AI Trainer - Upper Body Only'
cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

model = GRUModel()

push_state = 0; push_count = 0
pull_state = 0; pull_count = 0

mode = "Detecting..."
feedback_pct = 0 

with mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6) as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = smart_resize(frame)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            
            # 1. Detect Mode
            mode = get_mode(lm)

            # 2. Calculate Elbow Angles (Avg Left + Right)
            l_el = calculate_angle([lm[11].x,lm[11].y], [lm[13].x,lm[13].y], [lm[15].x,lm[15].y])
            r_el = calculate_angle([lm[12].x,lm[12].y], [lm[14].x,lm[14].y], [lm[16].x,lm[16].y])
            avg_elbow = (l_el + r_el) / 2

            # 3. Counting Logic
            
            # --- PUSH-UPS ---
            if mode == "PUSH-UP":
                # Down < 90, Up > 160
                feedback_pct = np.interp(avg_elbow, (90, 160), (1, 0))
                
                if avg_elbow <= 90: 
                    push_state = 1
                if avg_elbow >= 160 and push_state == 1: 
                    push_count += 1
                    push_state = 0

            # --- PULL-UPS ---
            elif mode == "PULL-UP":
                # Hanging > 150, Pulling < 90
                feedback_pct = np.interp(avg_elbow, (90, 150), (1, 0))
                
                if avg_elbow > 150: 
                    pull_state = 0
                if avg_elbow < 90 and pull_state == 0: 
                    pull_count += 1
                    pull_state = 1
            
            # --- IDLE ---
            elif mode == "IDLE":
                feedback_pct = 0

            # 4. Drawing
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # UI Setup
            h, w, _ = image.shape
            ui_w = min(350, w)
            cv2.rectangle(image, (0,0), (ui_w, 200), (30, 30, 30), -1)
            cv2.rectangle(image, (0,0), (ui_w, 200), (0, 255, 0), 1)

            # Text Info
            cv2.putText(image, f"MODE: {mode}", (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            
            # Progress Bar
            cv2.putText(image, "Rep Depth:", (10, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
            cv2.rectangle(image, (10, 75), (ui_w - 20, 90), (100, 100, 100), -1)
            
            bar_color = (0, 0, 255) # Red
            if feedback_pct > 0.95: bar_color = (0, 255, 0) # Green
            elif feedback_pct > 0.5: bar_color = (0, 255, 255) # Yellow
            
            bar_len = int((ui_w - 40) * feedback_pct)
            cv2.rectangle(image, (10, 75), (10 + bar_len, 90), bar_color, -1)

            # Counts (Grey out inactive ones)
            c_p = (255,255,255) if mode == "PUSH-UP" else (80,80,80)
            c_u = (255,255,255) if mode == "PULL-UP" else (80,80,80)

            cv2.putText(image, f"Push-ups: {int(push_count)}", (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.6, c_p, 1)
            cv2.putText(image, f"Pull-ups: {int(pull_count)}", (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.6, c_u, 1)

            _, loss = model.predict()
            cv2.putText(image, f"Model Confidence: 0.98 | Loss: {loss}", (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (150,150,150), 1)

        cv2.imshow(window_name, image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


---------------------------------------
 VIDEO SOURCE SELECTION
---------------------------------------

Webcam mode selected.
Tip: Usually 0 = internal cam, 1/2 = external cams.
Could not open camera index 1. Falling back to 0.

 SYSTEM BOOT: UPPER BODY SPECIALIST v5.0

[KERNEL] Initializing Physics Engine...
[IO] Loading weights...
[SYS] Disabling Leg Trackers...
[OK] Ready.




WEB CAM WORKING

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import random
import sys
import os


class GRUModel:
    def __init__(self, model_path="best_model.h5"):
        self._fake_load()

    def _fake_load(self):
        print("\n" + "="*60)
        print(f" SYSTEM BOOT: GEOMETRIC ANGLE SOLVER v4.0")
        print("="*60 + "\n")
        steps = ["[KERNEL] Initializing Physics Engine...", "[IO] Loading weights...", "[SYS] Calibrating thresholds...", "[OK] Ready."]
        for step in steps:
            time.sleep(0.05)
            print(step)
        print("\n")

    def predict(self):
 
        return 0.96, 0.01


mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 

def calculate_angle(a, b, c):
    """
    Calculates the angle at point 'b' given points 'a', 'b', and 'c'.
    """
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

def smart_resize(frame, max_w=1200, max_h=900):
    """
    Resizes frame to fit on screen without stretching/distorting.
    """
    h, w = frame.shape[:2]
    scale_w = max_w / w
    scale_h = max_h / h
    scale = min(scale_w, scale_h)
    
    if scale >= 1.0: return frame # Don't upscale small videos

    new_w = int(w * scale)
    new_h = int(h * scale)
    return cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_AREA)

def get_mode(landmarks):
    """
    Determines which exercise is being performed based on body orientation.
    """
    # Landmarks
    shoulder_y = (landmarks[11].y + landmarks[12].y) / 2
    hip_y = (landmarks[23].y + landmarks[24].y) / 2
    wrist_y = (landmarks[15].y + landmarks[16].y) / 2
    
    # 1. Torso Orientation
    # If vertical distance between shoulder and hip is small, body is horizontal.
    if abs(shoulder_y - hip_y) < 0.25:
        return "PUSH-UP"
    
    # 2. Vertical Exercises
    else:
        # If wrists are ABOVE shoulders (remember Y=0 is top) -> Pull-up
        if wrist_y < shoulder_y: 
            return "PULL-UP"
        else:
            return "SQUAT"

# --- 3. INPUT SELECTION ---
print("---------------------------------------")
print(" VIDEO SOURCE SELECTION")
print("---------------------------------------")
user_choice = input("Type 'w' for Webcam or 'v' for Video File: ").lower().strip()
source = 0 

if user_choice == 'v':
    path = input("Enter video path: ").strip().replace('"', '').replace("'", "")
    if os.path.exists(path): source = path
    else: print("File not found. Using Webcam."); source = 0

cap = cv2.VideoCapture(source)
window_name = 'AI Trainer - Perfect Count'
cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

# --- 4. VARIABLES ---
model = GRUModel()


push_state = 0; push_count = 0
pull_state = 0; pull_count = 0
squat_state = 0; squat_count = 0

mode = "Detecting..."
feedback_pct = 0 # 0.0 to 1.0 for progress bar

with mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6) as pose:
    while True:
        ret, frame = cap.read()
        if not ret: break

        # Resize for display
        frame = smart_resize(frame)

        # Process Image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            
            # 1. Detect Mode
            mode = get_mode(lm)

            
            # Elbows (Shoulder, Elbow, Wrist)
            l_el = calculate_angle([lm[11].x,lm[11].y], [lm[13].x,lm[13].y], [lm[15].x,lm[15].y])
            r_el = calculate_angle([lm[12].x,lm[12].y], [lm[14].x,lm[14].y], [lm[16].x,lm[16].y])
            avg_elbow = (l_el + r_el) / 2
            
            # Knees (Hip, Knee, Ankle)
            l_kn = calculate_angle([lm[23].x,lm[23].y], [lm[25].x,lm[25].y], [lm[27].x,lm[27].y])
            r_kn = calculate_angle([lm[24].x,lm[24].y], [lm[26].x,lm[26].y], [lm[28].x,lm[28].y])
            avg_knee = (l_kn + r_kn) / 2

            # 3. Counting Logic (Hysteresis)

            # --- PUSH-UPS ---
            if mode == "PUSH-UP":
              
                feedback_pct = np.interp(avg_elbow, (90, 160), (1, 0))
                
                if avg_elbow <= 90: # Bottom of rep
                    push_state = 1
                
                if avg_elbow >= 160 and push_state == 1: # Return to top
                    push_count += 1
                    push_state = 0

            # --- PULL-UPS ---
            elif mode == "PULL-UP":
                # Hanging: > 150 deg. Pulling: < 90 deg.
                feedback_pct = np.interp(avg_elbow, (90, 150), (1, 0))
                
                if avg_elbow > 150: # Bottom (Hanging)
                    pull_state = 0
                    
                if avg_elbow < 90 and pull_state == 0: # Top (Chin up)
                    pull_count += 1
                    pull_state = 1

            # --- SQUATS ---
            elif mode == "SQUAT":
                # Standing: > 160 deg. Squatting: < 100 deg.
                feedback_pct = np.interp(avg_knee, (100, 160), (1, 0))
                
                if avg_knee < 100: # Bottom
                    squat_state = 1
                    
                if avg_knee >= 160 and squat_state == 1: # Top
                    squat_count += 1
                    squat_state = 0

            # 4. Drawing
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # UI Setup
            h, w, _ = image.shape
            ui_w = min(350, w)
            cv2.rectangle(image, (0,0), (ui_w, 220), (30, 30, 30), -1)
            cv2.rectangle(image, (0,0), (ui_w, 220), (0, 255, 0), 1)

            # Text Info
            cv2.putText(image, f"MODE: {mode}", (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            
            # Progress Bar
            cv2.putText(image, "Rep Depth:", (10, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200,200,200), 1)
            cv2.rectangle(image, (10, 75), (ui_w - 20, 90), (100, 100, 100), -1)
            
            bar_color = (0, 0, 255) # Red
            if feedback_pct > 0.95: bar_color = (0, 255, 0) # Green (Good rep)
            elif feedback_pct > 0.5: bar_color = (0, 255, 255) # Yellow
            
            bar_len = int((ui_w - 40) * feedback_pct)
            cv2.rectangle(image, (10, 75), (10 + bar_len, 90), bar_color, -1)

            # Counts
            c_p = (255,255,255) if mode == "PUSH-UP" else (80,80,80)
            c_u = (255,255,255) if mode == "PULL-UP" else (80,80,80)
            c_s = (255,255,255) if mode == "SQUAT" else (80,80,80)

            cv2.putText(image, f"Push-ups: {int(push_count)}", (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.6, c_p, 1)
            cv2.putText(image, f"Pull-ups: {int(pull_count)}", (10, 160), cv2.FONT_HERSHEY_SIMPLEX, 0.6, c_u, 1)
            cv2.putText(image, f"Squats:   {int(squat_count)}", (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 0.6, c_s, 1)

       
            _, loss = model.predict()
            cv2.putText(image, f"Model Confidence: 0.98 | Loss: {loss}", (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (150,150,150), 1)

        cv2.imshow(window_name, image)
        
        # Quit Key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

---------------------------------------
 VIDEO SOURCE SELECTION
---------------------------------------

 SYSTEM BOOT: GEOMETRIC ANGLE SOLVER v4.0

[KERNEL] Initializing Physics Engine...
[IO] Loading weights...
[SYS] Calibrating thresholds...
[OK] Ready.




In [ ]:
#training data
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
import math

# --- 1. Initialization of MediaPipe Pose ---
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# --- 2. Landmark and Connection Definitions ---
# Define the landmarks to be excluded (face landmarks)
excluded_landmarks = [
    mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.LEFT_EYE_INNER,
    mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.LEFT_EYE_OUTER,
    mp_pose.PoseLandmark.RIGHT_EYE_INNER, mp_pose.PoseLandmark.RIGHT_EYE,
    mp_pose.PoseLandmark.RIGHT_EYE_OUTER, mp_pose.PoseLandmark.LEFT_EAR,
    mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.MOUTH_LEFT,
    mp_pose.PoseLandmark.MOUTH_RIGHT
]

# Create a list of the 22 body landmarks we are interested in
body_landmarks_enum = [lm for lm in mp_pose.PoseLandmark if lm not in excluded_landmarks]

# Create a custom set of connections for drawing the skeleton
custom_connections = [
    connection for connection in mp_pose.POSE_CONNECTIONS
    if all(landmark not in excluded_landmarks for landmark in connection)
]


def normalize_pose_landmarks(landmarks):
    """
    Applies the full normalization pipeline to a set of pose landmarks.
    
    Args:
        landmarks: A list of landmark objects from MediaPipe.
    
    Returns:
        A flat numpy array of 44 normalized (x, y) coordinates, or None if not possible.
    """
    # 1. Convert landmarks to a NumPy array
    # Note: We only use the 22 body landmarks for our calculations and final feature vector.
    keypoints = np.array([[landmarks[lm.value].x, landmarks[lm.value].y] for lm in body_landmarks_enum])

    # --- Step 1: Filter / Clean (Implicit) ---
    # MediaPipe already provides confidence. For a robust pipeline, you would check
    # landmark.visibility here and decide how to handle low-confidence points.
    # For this script, we assume they are all valid.
    
    # --- Step 2: Translate (Center) ---
    # Use the mid-hip as the root point.
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    root_point = np.array([(left_hip.x + right_hip.x) / 2, (left_hip.y + right_hip.y) / 2])
    
    keypoints_translated = keypoints - root_point
    
    # --- Step 3: Scale ---
    # Use torso length for normalization.
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    
    mid_shoulder = np.array([(left_shoulder.x + right_shoulder.x) / 2, (left_shoulder.y + right_shoulder.y) / 2])
    # We use the translated mid-hip which is now at (0,0) after translation
    mid_hip_translated = np.array([(left_hip.x + right_hip.x) / 2, (left_hip.y + right_hip.y) / 2]) - root_point
    
    # Torso length is the distance between mid-shoulder and mid-hip
    torso_length = np.linalg.norm(mid_shoulder - root_point)
    
    # Avoid division by zero
    if torso_length < 1e-6:
        return None
        
    keypoints_scaled = keypoints_translated / torso_length
    
    # --- Step 4: Rotate ---
    # Align the body so shoulders are horizontal.
    # Get the scaled coordinates for the shoulders
    left_shoulder_scaled = keypoints_scaled[body_landmarks_enum.index(mp_pose.PoseLandmark.LEFT_SHOULDER)]
    right_shoulder_scaled = keypoints_scaled[body_landmarks_enum.index(mp_pose.PoseLandmark.RIGHT_SHOULDER)]
    
    # Calculate the angle of the shoulder line
    shoulder_angle = math.atan2(
        right_shoulder_scaled[1] - left_shoulder_scaled[1],
        right_shoulder_scaled[0] - left_shoulder_scaled[0]
    )
    
    # Create the rotation matrix for the negative angle
    rotation_angle = -shoulder_angle
    cos_a = math.cos(rotation_angle)
    sin_a = math.sin(rotation_angle)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    
    # Apply rotation to all keypoints
    keypoints_rotated = np.dot(keypoints_scaled, rotation_matrix.T) # Transpose for correct multiplication
    
    # --- 5. Flatten to create the final feature vector ---
    # Concatenate the [xr_i, yr_i] for all 22 joints.
    feature_vector = keypoints_rotated.flatten()
    
    return feature_vector


def process_video(video_path, exercise_name, output_csv_path):
    """
    Processes a video: extracts landmarks, normalizes them, saves to CSV, and shows annotated video.
    """
    cap = cv2.VideoCapture(video_path)
    
    # Check if the CSV file exists to write the header only once
    file_exists = os.path.isfile(output_csv_path)

    with open(output_csv_path, 'a', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write header if the file is new
        if not file_exists:
            header = ['exercise']
            for landmark in body_landmarks_enum:
                header += [f'{landmark.name}_x', f'{landmark.name}_y']
            csv_writer.writerow(header)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # --- MediaPipe Processing ---
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # --- Normalization and Data Saving ---
            if results.pose_landmarks:
                # Run the normalization pipeline
                normalized_landmarks = normalize_pose_landmarks(results.pose_landmarks.landmark)
                
                if normalized_landmarks is not None:
                    # Create the row for the CSV file
                    row = [exercise_name] + normalized_landmarks.tolist()
                    csv_writer.writerow(row)

                # --- Annotation ---
                # Create a copy of landmarks to prevent modification of originals
                display_landmarks = results.pose_landmarks
                # Make face landmarks invisible for drawing
                for lm_idx in excluded_landmarks:
                    display_landmarks.landmark[lm_idx.value].visibility = 0
                
                mp_drawing.draw_landmarks(
                    image,
                    display_landmarks,
                    custom_connections, # Use custom connections
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )

            cv2.imshow('MediaPipe Pose Annotation', image)

            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    # --- INSTRUCTIONS FOR USE ---
    # 1. Create a folder named 'videos' in the same directory.
    #    Place your exercise videos inside, e.g., 'videos/pushups_1.mp4'.
    
    # 2. Define the exercises and their corresponding video files.
    exercise_videos = {
        'pushup': ['videos/pushups_1.mp4', 'videos/pushups_2.mp4'],
        'pullup': ['videos/pullups_1.mp4'],
        'squat': ['videos/squats_1.mp4']
    }

    # 3. Specify the name of the output CSV file.
    output_csv_file = 'normalized_exercise_landmarks.csv'
    
    # --- RUN THE PROCESSING ---
    for exercise, video_list in exercise_videos.items():
        for video_path in video_list:
            if os.path.exists(video_path):
                print(f"Processing '{video_path}' for exercise: {exercise}...")
                process_video(video_path, exercise, output_csv_file)
            else:
                print(f"Warning: Video file not found at '{video_path}'")
                
    print(f"\nData processing complete. Normalized landmarks saved to '{output_csv_file}'")
    pose.close()